In [0]:
# Mount Google Drive
%%capture
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive

drive.mount(ROOT);           # we mount the google drive at /content/drive

In [0]:
# Set working directory
%%capture
%cd /content/drive/My Drive/restoration-mapper/task_driven_data_augmentation

In [0]:
'''
This script is to preprocess all the data into the chosen resolution and fixed dimensions specified in the init_acdc.py config file.
This re-sampled (to target resolution) and cropped/zero-padded image,label pairs are stored in npy files and are later used directly while the training of the network.
'''
import numpy as np
import pathlib

In [0]:
class params:
  dataset = 'acdc'
  #no of training images
  no_of_tr_imgs = 'tr3' # Options include: ['tr1', 'tr3', 'tr5', 'tr15', 'tr40']
  #combination of training images
  comb_tr_imgs = 'c1' # Options include: ['c1', 'c2', 'c3', 'c4', 'c5']

  #learning rate of seg unet
  lr_seg = 0.001
  # learning rate of generator
  lr_gen = 0.0001
  # learning rate of discriminator
  lr_disc = 0.0001
  # lat dim of z sample
  z_lat_dim = 100

  # ra_en : 0 - disabled, 1 - enabled
  ra_en = 0
  # select gan type
  gan_type = 'gan' # Options include: ['lsgan', 'gan', 'wgan-gp','ngan']
  # beta value of Adam optimizer
  beta_val = 0.9
  # to enable the representation of labels with 1 hot encoding
  en_1hot = 1

  # lamda factors
  # for segmenation loss term (lamda_dsc)
  lamda_dsc = 1
  # adversarial loss term (lamda_adv)
  lamda_adv = 1
  ### deformation field cGAN specific
  # for negative L1 loss on spatial transformation (per-pixel flow field/deformation field) term (lamda_l1_g)
  lamda_l1_g = 0.001

  ### Intensity field cGAN specific
  # for negative L1 loss on transformation (additive intensity field) term (lamda_l1_i)
  lamda_l1_i = 0.001

  #version of run
  ver = 0

  #data aug - 0 - disabled, 1 - enabled
  data_aug_seg = 1 # Options include: [0,1]

  # segmentation loss to optimize
  # 0 for weighted cross entropy, 1 for dice score loss
  dsc_loss = 0 # Options include: [0,1]

In [0]:
######################################
# create cropped images
# ####################################
if params.dataset == 'acdc':
    #print('load acdc configs')
    import experiment_init.init_acdc as cfg
    import experiment_init.data_cfg_acdc as data_list
else:
    raise ValueError(params.dataset)

from dataloaders import dataloaderObj
dt = dataloaderObj(cfg)

if params.dataset == 'acdc':
    #print('set acdc orig img dataloader handle')
    orig_img_dt=dt.load_acdc_imgs

#loop over all image,label pairs to create cropped image,label pairs
for index in range(1,101):
    # Load each image
    if(index<10):
        test_id='00'+str(index)
    elif(index<100):
        test_id='0'+str(index)
    else:
        test_id=str(index)
    test_id_l=[test_id]

    #load image,label pairs and process it to chosen resolution and dimensions    
    img_sys,label_sys,pixel_size,affine_tst= orig_img_dt(test_id_l,ret_affine=1)
    cropped_img_sys,cropped_mask_sys = dt.preprocess_data(img_sys, label_sys, pixel_size)

    #save the processed cropped img and its label
    save_dir_tmp=str(cfg.data_path_tr_cropped)+str(test_id)+'/'
    pathlib.Path(save_dir_tmp).mkdir(parents=True, exist_ok=True)
    savefile_name=str(save_dir_tmp)+'img_cropped.npy' 
    np.save(savefile_name,cropped_img_sys)
    savefile_name=str(save_dir_tmp)+'mask_cropped.npy' 
    np.save(savefile_name,cropped_mask_sys)